In [ ]:
include("optimization_library.jl");

# Exercise 7.1: Constraint Elimination

In [ ]:
# Solving a problem with equality constraints Ax = b using constraint elimination and interior point method

# f: Objective function
# df: Gradient vector of the objective function
# Hf: Hessian matrix of the objective function
# g: Array of inequality constraints g_i(x) < 0
# dg: Array with the gradient of the inequality constraints
# Hg: Array with the Hessian of the inequality constraints
# A: matrix for equality constraints Ax = b
# b: vector for equality constraints Ax = b
# barrier_increase: Factor by how much to increase the barrier in every outer iteration
# maxiters: maximum number of outer iterations
# inner_maxiters: maximum number of inner iterations (within descent method)
# eps: Stopping criterion

function ConstraintElimination_ipm(f,df,Hf,A,b;g=[],dg=[],Hg=[],eps = 0.0001, barrier_increase = 2.0,
                                    maxiters = 100, inner_maxiters = 100)

    nx = size(A,2) # dimensions of the original problem
    nz = nx - LA.rank(A) # dimensions of the unconstrained problem
    
    # vectors for transformed positivity constraints --> entry gg[i] is a function depending on z (variable of the
    # problem where equality constraints are removed)
    
    ng = size(g,1)
    gg = Vector(undef, ng)
    dgg = Vector(undef, ng)
    Hgg = Vector(undef, ng)
    
    # trace of interior point method
    trace = 1
    
# ===============================================================================
    
    # 1. Find an initial admissible solution for, the equality constraints
    x0 = A \ b
    
    # 2. Construct the matrix F (see lecture 4 for the notation)
    F = LA.nullspace(A)
    
    # 3. Construct new objective functions, its gradient and Hessian 
    # Concerning the gradient and Hessian: Beware that you derive for z and have to respect the chain rule!
    obj(z) = f(F * z .+ x0)
    dobj(z) = F' * df(F * z .+ x0)
    Hobj(z) = F' * Hf(F * z .+ x0) * F
    
    # 4. Transform the inequality constraints, their gradients and Hessians to be functions of Fz + x0
    for i in 1:ng
        gg[i] = z -> g[i](F * z + x0)
        dgg[i] = z -> F' * dg[i](F * z + x0)
        Hgg[i] = z -> F' * Hg[i](F * z + x0) * F
    end

    # 5. Randomly initialize z and find an admissible interior point for the transformed problem
    z0 = rand(nz)
    z_adm = ipm_admissible(z0; g=gg, dg=dgg, maxiters = 1000, eps=eps, inner_maxiters = 100,
                               barrier_increase = barrier_increase)
    
    # 6. use the admissible interior point to solve the transformed problem
    zstar, trace = ipm(obj, dobj, Hobj, z_adm, g = gg, dg = dgg, Hg = Hgg, barrier_increase = barrier_increase)
    
    # 7. reconstruct the solution of the original problem
    xstar = F * zstar + x0
    
# ===============================================================================
    return xstar, trace
    
end

# Exercise 7.2: Santa's Optimal Transport Problem

<img src="figures/santa-claus.png" width="300">

In [ ]:
# children
child_nz = 0.8
child_eu = 78
child_usa = 60
child_bz = 40;

In [ ]:
child_total = child_bz + child_eu + child_nz + child_usa

In [ ]:
p_an = 0.5*child_total
p_gr = 0.3*child_total
p_al = 0.2*child_total

In [ ]:
A = [1 0 0 0 1 0 0 0 1 0 0 0;
     0 1 0 0 0 1 0 0 0 1 0 0;
     0 0 1 0 0 0 1 0 0 0 1 0;
     0 0 0 1 0 0 0 1 0 0 0 1;
     1 1 1 1 0 0 0 0 0 0 0 0;
     0 0 0 0 1 1 1 1 0 0 0 0;
     0 0 0 0 0 0 0 0 1 1 1 1];

In [ ]:
b = [child_eu; child_usa; child_bz; child_nz; p_al; p_an; p_gr]

In [ ]:
# Antarktika
## New Zealand: 4.868 km
## USA: 14.601 km
## Europe: 16.396 km
## Brazil: 8.344 kilometers

# Alaska
## New Zealand: 12.089 km
## USA: 5380 KM 
## Europe: 6.744 km
## Brazil: 11.788 km


# Greenland 
## New Zealand: 16.012 km
## USA: 4.857 km
## Europe: 3.281 km
## Brazil: 9 560 kilometers

# costs, i.e. distances
xi = vec([6744 16396 3281;
          5380 14601 4857;
          11788 8344 9560;
          12089 4868 16012
        ]);

In [ ]:
nx = size(A,2)

f(x) = xi' * x
df(x) = xi
Hf(x) = zeros(nx,nx)

g = Vector(undef, nx)
dg = Vector(undef, nx)
Hg = Vector(undef, nx)
    
for i in 1:nx # allow only positive quantities
    g[i] = x -> -x[i]
    dg[i] = x -> begin d = zeros(nx); d[i] = -1; return d end
    Hg[i] = x -> zeros(nx, nx)
end

In [ ]:
x_best, trace = ConstraintElimination_ipm(f,df,Hf,A,b,g=g,dg = dg, Hg = Hg, eps = 0.0001, barrier_increase = 10.0)
round.(reshape(x_best, (4,3)),digits = 4)